In [4]:
from sagemaker.processing import ScriptProcessor, ProcessingInput, ProcessingOutput
import sagemaker
from sagemaker.estimator import Estimator
from sagemaker.experiments.run import Run, load_run
import time
from datetime import datetime
from sagemaker.inputs import TrainingInput
from sagemaker import image_uris

sagemaker.__version__

'2.219.0'

In [5]:
data_version = "v1"
model_version = "v1"
session = sagemaker.Session()
sm = session.sagemaker_client

experiment_name = "team1-index-predictor-model-training"
trial_suffix = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
trial_name = f"model-training-{model_version}-{trial_suffix}"

bucket_name = "team1-index-predictor-bucket"
input_prefix = "data/processed"
output_bucket_prefix = "models"

processing_instance_type = "ml.t3.medium"
processing_instance_count = 1

In [6]:
with Run(
    experiment_name=experiment_name,
    run_name=trial_name,
    run_display_name=trial_name,
    sagemaker_session=session,
) as run:
    experiment_config = run.experiment_config

xgboost_container = image_uris.retrieve("xgboost", session.boto_region_name, version="1.3-1")

estimator = Estimator(
    image_uri=xgboost_container,
    role=sagemaker.get_execution_role(),
    instance_count=1,
    instance_type='ml.m5.large',
    output_path=f"s3://{bucket_name}/{output_bucket_prefix}",
    sagemaker_session=sagemaker.Session(),
    entry_point='../src/models/trainer.py',
)

estimator.set_hyperparameters(
    input_path="/opt/ml/input/data/train",
    data_version=data_version,
    target_column='Close_target',
    model_output_path="/opt/ml/model/model",
    num_trials=10
)

train_input = TrainingInput(s3_data=f"s3://{bucket_name}/{input_prefix}")

estimator.fit({'train': train_input})

INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2024-06-18-23-40-03-184


2024-06-18 23:40:03 Starting - Starting the training job...
2024-06-18 23:40:21 Starting - Preparing the instances for training...
2024-06-18 23:40:48 Downloading - Downloading input data...
2024-06-18 23:41:34 Downloading - Downloading the training image......
2024-06-18 23:42:09 Training - Training image download completed. Training in progress...[2024-06-18 23:42:14.663 ip-10-0-127-133.eu-central-1.compute.internal:7 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2024-06-18 23:42:14.687 ip-10-0-127-133.eu-central-1.compute.internal:7 INFO profiler_config_parser.py:111] User has disabled profiler.
[2024-06-18:23:42:14:INFO] Imported framework sagemaker_xgboost_container.training
[2024-06-18:23:42:14:INFO] Failed to parse hyperparameter data_version value v1 to Json.
Returning the value itself
[2024-06-18:23:42:14:INFO] Failed to parse hyperparameter input_path value /opt/ml/input/data/train to Json.
Returning the value itself
[2024-06-18:23:42:14:INFO] Failed to parse hyperpa

In [10]:
!aws s3 ls s3://{bucket_name}/{output_bucket_prefix}/model.xgb/sagemaker-xgboost-2024-06-18-23-40-03-184/output/model.tar.gz

2024-06-18 23:44:15      72618 model.tar.gz


In [54]:
import gc
gc.collect()

24255

In [11]:
!aws s3 cp s3://{bucket_name}/{output_bucket_prefix}/model.xgb/sagemaker-xgboost-2024-06-18-23-40-03-184/output/model.tar.gz model.tar.gz

download: s3://team1-index-predictor-bucket/models/model.xgb/sagemaker-xgboost-2024-06-18-23-40-03-184/output/model.tar.gz to ./model.tar.gz
